In [1]:
import os
import json
from qiskit import IBMQ

this_dir = os.getcwd()

program_path = os.path.join(this_dir, 'schwinger_rqd.py')

In [2]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research', group='tokyo-1', project='main')
#provider = IBMQ.get_provider(hub='ibm-q-utokyo', group='internal', project='icepp')

## Write the program file

In [3]:
output = open(program_path, 'w')
main_source = open(os.path.join(this_dir, 'main.py'))

while True:
    line = main_source.readline()
    if not line:
        break
        
    if line.startswith('## include'):
        nextline = main_source.readline()
        if nextline.startswith('from ') or nextline.startswith('import '):
            module = nextline.split()[1]
        else:
            raise RuntimeError('Invalid include block: {}'.format(nextline.strip()))
            
        include_path = os.path.join(line.split()[2], *module.split('.')) + '.py'
        
        with open(include_path) as source:
            output.write(source.read())
            output.write('\n')
            
    elif line.startswith('## ignore'):
        main_source.readline()
    else:
        output.write(line)
    
output.close()
main_source.close()

## Set the metadata dict

In [4]:
version = 6

metadata = {
    'name': 'schwinger-rqd-{}'.format(version),
    'description': 'Restarted Quantum Dynamics simulation of the lattice Schwinger model.',
    'max_execution_time': 14400,
    'version': float(version),
    'backend_requirements': {'min_num_qubits': 4},
    'parameters': [
        {'name': 'api_token', 'description': 'IBM Quantum API token.', 'type': 'str', 'required': True},
        {'name': 'num_sites', 'description': 'Number of lattice sites (qubits).', 'type': 'int', 'required': True},
        {'name': 'aJ', 'description': 'Square of the product of the coupling constant and the lattice spacing, divided by 2.', 'type': 'float', 'required': True},
        {'name': 'am', 'description': 'Mass times the lattice spacing.', 'type': 'float', 'required': True},
        {'name': 'omegadt', 'description': 'Trotter time step divided by double the lattice spacing.', 'type': 'float', 'required': True},
        {'name': 'num_tsteps', 'description': 'Number of Trotter steps to simulate.', 'type': 'int', 'required': True},
        {'name': 'tsteps_per_rqd', 'description': 'Number of Trotter steps to advance in each RQD step.', 'type': 'int', 'required': True},
        {'name': 'physical_qubits', 'description': 'List of physical qubits to use.', 'type': 'list', 'required': False},
        {'name': 'error_matrix', 'description': 'Measurement error matrix to be passed to the MeasurementFilter.', 'type': 'ndarray', 'required': False},
        {'name': 'max_sweeps', 'description': 'Number of parameter sweeps for sequential minimum optimization.', 'type': 'int', 'required': False},
        {'name': 'minimizer_shots', 'description': 'Number of shots to use for sequential minimum optimization.', 'type': 'int', 'required': False},
        {'name': 'forward_shots', 'description': 'Number of shots to use for the forward steps of RQD.', 'type': 'int', 'required': False},
        {'name': 'resume_from', 'description': 'Minimizer parameters to resume RQD.', 'type': 'dict', 'required': False}
    ],
    'return_values': [
        {'name': 'forward_counts', 'description': 'List of counts dictionaries of the Trotter steps.', 'type': 'list'},
        {'name': 'optimal_params_list', 'description': 'List of ansatz parameter values that approximates the Trotter simulation targets.', 'type': 'list'}
    ],
    'interim_results': [
        {'name': 'rqd_step', 'description': 'RQD iteration number.', 'type': 'int'},
        {'name': 'forward_counts', 'description': 'List of counts dictionaries of the Trotter steps in one RQD step.', 'type': 'list'},
        {'name': 'optimal_params', 'description': 'Optimal ansatz parameter values from the previous RQD step.', 'type': 'ndarray'},
        {'name': 'isweep', 'description': 'Current sweep number of the minimizer.', 'type': 'int'},
        {'name': 'param_val', 'description': 'Ansatz parameter values at sweep number isweep.', 'type': 'ndarray'},
        {'name': 'cost', 'description': 'Value of the cost function at sweep number isweep.', 'type': 'float'},
        {'name': 'total_shots', 'description': 'Number of shots used so far.', 'type': 'int'},
        {'name': 'shots_values', 'description': 'Array of cumulative shot numbers during the RQD iteration', 'type': 'ndarray'},
        {'name': 'cost_values', 'description': 'Array of the cost value evolution during the RQD iteration', 'type': 'ndarray'}
    ]
}

## Upload the program

In [5]:
try:
    provider.runtime.delete_program('schwinger-rqd-{}'.format(version - 1))
except:
    pass

program_id = provider.runtime.upload_program(
    data=program_path,
    metadata=metadata
)
print('Uploaded program with id', program_id)

Uploaded program with id schwinger-rqd-6
